# Inspecting intertexts in a micro level

## Loading in the needed librarires

In [1]:
import glob
import os
from collections import Counter
from itertools import combinations, product

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import seaborn as sb

from sklearn.metrics import pairwise_distances
import lxml.etree
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'Arial'
from IPython.display import display

## Loading in data and metadata

In [2]:
meta_df = pd.read_excel('../data/metadata_corrected.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,provenance,date_range,genre,subgenre,exclude
49,derde_martijn,Derde Martijn,Jacob van Maerlant,cdrom-mnl,1380-1425,Lyriek,NaN,NaN
206,sinte_kunera,Sinte Kunera,NaN,cdrom-mnl,1400-1500,Epiek,Heiligenleven,NaN
59,esopet,Esopet,Calfstaf en Noydekin,cdrom-mnl,1340-1360,Epiek,Dier,NaN
235,spiegel_historiael__4_velthem__fragm_we,Spiegel historiael (P4),Lodewijk van Velthem,cdrom-mnl,1393-1402,Epiek,Historiografie,NaN
48,der_ystorien_bloeme,Der ystorien bloeme,NaN,cdrom-mnl,1340-1360,Epiek,Historiografie,NaN
191,roman_van_caesar_fragm_br,Roman van Caesar,NaN,cdrom-mnl,1300-1350,Epiek,Ridder,NaN
247,truwanten,Truwanten,NaN,cdrom-mnl,1400-1420,Dramatiek,NaN,NaN
60,vanden_vier_vingheren_ende_vanden_dume,Expositie vanden viere vingheren ende vanden dume,NaN,cdrom-mnl,1400-1420,Epiek,Didactiek,NaN
70,Floyris ende Blantseflur,Floyris ende Blantseflur,NaN,CG1,1225-1201,Epiek,Ridder,NaN
65,florigout_fragm_l,Florigout,NaN,cdrom-mnl,1375-1400,Epiek,Ridder,NaN


In [3]:
def get_verse_groups(verses, size=2, intertexts=False):
    for i in range(len(verses) - (size - 1)):
        if not intertexts:
            yield ' / '.join(verses[i : i + size])
        else:
            its = Counter(verses[i : i + size])
            if None in its:
                yield None
            elif len(its) > 1:
                yield 'overlap'
            else:
                yield list(its.keys())[0]

In [4]:
def parse_xml(fn, rm_interpol=True, rm_prose=True):
    try:
        tree = lxml.etree.parse(fn)
    except OSError:
        print(f'- Could not load {fn}')
        return None
        
    if rm_interpol:
        for interpolation in tree.xpath("//interpolation"):
            interpolation.getparent().remove(interpolation)

    if rm_prose:
        for prose in tree.xpath("//prose"):
            prose.getparent().remove(prose)

        
    for line_node in tree.iterfind('//l'):
        try:
            intertext_id = line_node.attrib['intertext']
        except KeyError:
            intertext_id = None
        
        tokens_ = line_node.attrib['tokens'].split()
        lemmas_ = []
        
        lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
        pos_tags = [p.text for p in line_node.iterfind('.//pos')]
        
        for lemma, pos in zip(lemma_tags, pos_tags):
            for l, p in zip((lemma or '').split('+'), (pos or '').split('+')):
                    if p == 'NOU-P':
                        lemmas_.append('NOU-P')
                    else:
                        lemmas_.append(l)
    
        yield tokens_, lemmas_, intertext_id

In [5]:
GROUP_SIZE = 2

titles, tokens, lemmas, intertexts = [], [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas, work_intertexts = [], [], []
    
    for id_ in sorted(group['id']):
        for tok, lem, intertext_id in parse_xml(f'../data/xml_galahad_BY-split/{id_}.xml'):
            work_tokens.append(tok)
            work_lemmas.append(lem)
            work_intertexts.append(intertext_id)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))
    verse_group_intertexts = list(get_verse_groups(work_intertexts, size=GROUP_SIZE, intertexts=True))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    intertexts.extend(verse_group_intertexts)
    titles.extend([title] * len(verse_group_lemmas))

100%|█████████████████████████████████████████| 208/208 [00:28<00:00,  7.41it/s]


In [6]:
df = pd.DataFrame(zip(titles, tokens, lemmas, intertexts), columns=('title', 'tokens', 'lemmas', 'intertext'))

In [7]:
df

,title,tokens,lemmas,intertext
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe haar tuut staan,None
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe haar tuut staan / klaar dat zij zij blanketten,None
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij blanketten / die kleed zo lang dat zij zij letten,None
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,die kleed zo lang dat zij zij letten / en slepen zij na al op die aarde,None
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slepen zij na al op die aarde / fi die vuil hovaart,None
...,...,...,...,...
861522,Wrake van Ragisel,Nu selewi swigen van desen / Ende van enen jongelinc vord lesen,nu zullen wij zwijgen van deze / en van een jongeling voort lezen,None
861523,Wrake van Ragisel,Ende van enen jongelinc vord lesen / Die te hove nu sal comen,en van een jongeling voort lezen / die te hof nu zullen komen,None
861524,Wrake van Ragisel,Die te hove nu sal comen / Eest alsict hebbe vernomen,die te hof nu zullen komen / zijn het als zij hebben vernemen,None
861525,Wrake van Ragisel,Eest alsict hebbe vernomen / Soe salmen noch van hem sien,zijn het als zij hebben vernemen / zo zullen men nog van hij zien,None


In [8]:
def tokenizer(text):
    return text.replace(' / ', ' ').lower().strip().split()

def add_rhyme_column(df):
    rhyme_words = []
    for lemmas in df['lemmas']:
        rhymes = []
        for verse in lemmas.split(' / '):
            words = verse.strip().split()
            if words:  # Ensure the verse is not empty
                rhymes.append(words[-1])
            else:
                rhymes.append('')  # Use an empty string or placeholder
        rhyme_words.append(' '.join(rhymes))
    df['rhyme'] = rhyme_words
    return df

In [9]:
df = add_rhyme_column(df)
df.head()

,title,tokens,lemmas,intertext,rhyme
0,AB recht ende averecht,Aensiet dese vrouwen hoe si gaen / Besiet hoe haer tuten staen,aanzien deze vrouw hoe zij gaan / bezien hoe haar tuut staan,None,gaan staan
1,AB recht ende averecht,Besiet hoe haer tuten staen / Claer dat si hen blanketten,bezien hoe haar tuut staan / klaar dat zij zij blanketten,None,staan blanketten
2,AB recht ende averecht,Claer dat si hen blanketten / Die cleeder soe lanc dat si hen letten,klaar dat zij zij blanketten / die kleed zo lang dat zij zij letten,None,blanketten letten
3,AB recht ende averecht,Die cleeder soe lanc dat si hen letten / Ende sleypen hen nae al op die eerde,die kleed zo lang dat zij zij letten / en slepen zij na al op die aarde,None,letten aarde
4,AB recht ende averecht,Ende sleypen hen nae al op die eerde / Fi diere vuylder hoverde,en slepen zij na al op die aarde / fi die vuil hovaart,None,aarde hovaart


In [10]:
optim_vs = 7000
optim_rw = 0.1693877551020408
optim_th = 0.42202047865566794

In [11]:
vec = TfidfVectorizer(max_features=optim_vs, min_df=2,
                      tokenizer=tokenizer, token_pattern=None).fit(df['lemmas'])

## Nearest neighbors

In [12]:
def nearest_verses(title1, title2, base_df, vectorizer, prop_filt=2, rhyme_weight=.15):
    A = base_df[base_df['title'] == title1]
    B = base_df[base_df['title'] == title2]
    
    AX = vec.transform(A['lemmas']) + rhyme_weight * vec.transform(A['rhyme'])
    BX = vec.transform(B['lemmas']) + rhyme_weight * vec.transform(B['rhyme'])
    
    nearest = []
    for i, ax in enumerate(AX):
        distances = pairwise_distances(ax, BX, metric='cosine').flatten()
        top_i = distances.argsort()[0]
        nearest.append((A.iloc[i]['tokens'], B.iloc[top_i]['tokens'],
                        A.iloc[i]['lemmas'], B.iloc[top_i]['lemmas'],
                        A.iloc[i]['title'], B.iloc[top_i]['title'],
                        A.iloc[i]['intertext'], B.iloc[top_i]['intertext'],
                        distances[top_i]))

    nearest = pd.DataFrame(nearest,
                           columns=['tokens1', 'tokens2', 'lemmas1', 'lemmas2',
                                    'title1', 'title2', 'intertext1', 'intertext2',
                                    'distance'])

    nearest = nearest[nearest['title1'] != nearest['title2']]
    nearest = nearest[
                      (nearest['lemmas1'].str.count('NOU-P') < prop_filt) & \
                      (nearest['lemmas1'].str.count('NOU-P') < prop_filt) 
                     ]
    return nearest

## Cases Studies

In [13]:
nearest_df = nearest_verses('Boec vander wraken', 'Sidrac',
                            base_df=df, vectorizer=vec, prop_filt=2, rhyme_weight=0)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/wraken-sidrac.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
5868,Nu seght allegader Amen / Amen,Amen segt gemeenlike / Amen,nu zeggen algader amen / amen,amen zeggen gemeenlijk / amen,Boec vander wraken,Sidrac,None,None,0.155842
12,Daer om so biddic met trouwen / Hen die dit boec selen scouwen,Ende ic bidde hem allen met trauwen / Die desen bouc selen scauwen,daar om zo bidden ik met trouw / zij die dit boek zullen schouwen,en ik bidden hij al met trouw / die deze boek zullen schouwen,Boec vander wraken,Sidrac,None,None,0.189886
5838,Amen seghet allegader Amen / Ay hoe menichwerven heb ic gheseit,Amen segt gemeenlike / Amen,amen zeggen algader amen / ai hoe menigwerf hebben ik zeggen,amen zeggen gemeenlijk / amen,Boec vander wraken,Sidrac,None,None,0.282951
1518,MI quam een out boec in de hant / Daer ic in ghescreven vant,Soe quam my een boec ter hant / Daer ic in bescreven vant,ik komen een oud boek in de hand / daar ik in schrijven vinden,zo komen ik een boek ter hand / daar ik in beschrijven vinden,Boec vander wraken,Sidrac,None,None,0.312443
5837,Des onne ons die hemelsche Vader / Amen seghet allegader Amen,Amen segt gemeenlike / Amen,des on ons die hemels vader / amen zeggen algader amen,amen zeggen gemeenlijk / amen,Boec vander wraken,Sidrac,None,None,0.338070
5083,Ghesteinte silver ende gout / Ende diere juwele menichfout,Die beter es menichfout / Danne ghesteente zelver of gout,gesteente zilver en goud / en die juweel menigvout,die goed zijn menigvout / dan gesteente zelf of goud,Boec vander wraken,Sidrac,None,None,0.361802
5464,Dats Cristus mijn Sone / Dien ic hebbe allene,Dat God ende ghi allene / Enen sone hebt gemene,dat zijn NOU-P mijn zoon / dat ik hebben alleen,dat NOU-P en gij alleen / een zoon hebben menen,Boec vander wraken,Sidrac,None,None,0.370025
5465,Dien ic hebbe allene / Met sinen Vader ghemene,Dat God ende ghi allene / Enen sone hebt gemene,dat ik hebben alleen / met zijn vader menen,dat NOU-P en gij alleen / een zoon hebben menen,Boec vander wraken,Sidrac,None,None,0.372586
2232,Armelus dese grote here / Minde die vrouwe so zere,Die dese vrouwe niet en mint / Die de Vader alsoe minde,NOU-P deze groot heer / minnen die vrouwe zo zeer,die deze vrouwe niet ne minnen / die de vader alzo minnen,Boec vander wraken,Sidrac,None,None,0.393477
2777,Dat selen die sijn waerlike / Die verdient hebben dat hemelrike,Dat overlijt soe cortelike / Verdienen souden hemelrike,dat zullen die zijn waarlijk / die verdienen hebben dat hemelrijk,dat overlijd zo kortelijk / verdienen zullen hemelrijk,Boec vander wraken,Sidrac,None,None,0.414915


In [14]:
nearest_df = nearest_verses('Der leken spieghel', 'Jans teesteye',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/lekenspiegel-teesteye.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
4699,Elc mensche die sal / Gode minnen bouen al,Elc mensche die sal / Gode minnen boven al,elk mens die zullen / NOU-P minnen boven al,elk mens die zullen / NOU-P minnen boven al,Der leken spieghel,Jans teesteye,None,None,0.000000e+00
4708,Ende oec niet stelen wats ghesciet / Nieman doeden noch verslaen,Ende oec niet stelen wats ghesciet / Nieman doden noch verslaen,en ook niet stelen wat geschieden / niemand doden noch verslaan,en ook niet stelen wat geschieden / niemand doden noch verslaan,Der leken spieghel,Jans teesteye,None,None,0.000000e+00
3501,Sal men gaen daer toe / Wijslijc ende met maten,Salmen gaen daer toe / Wijsselijc ende met maten,zullen men gaan daar toe / wijselijk en met maat,zullen men gaan daar toe / wijselijk en met maat,Der leken spieghel,Jans teesteye,None,None,1.110223e-16
4709,Nieman doeden noch verslaen / Gheen valsch orconde doen verstaen,Nieman doden noch verslaen / Gheen valsche orconde doen verstaen,niemand doden noch verslaan / geen vals oorkonde doen verstaan,niemand doden noch verslaan / geen vals oorkonde doen verstaan,Der leken spieghel,Jans teesteye,None,None,3.330669e-16
2732,Want soe die nideghe langher leeft / Soe hi meer onwils heeft,Want so die nideghe langher leeft / So hi meer sijns onwillen heeft,want zo die nijdige lang leven / zo hij meer onwil hebben,want zo die nijdige lang leven / zo hij meer zijn onwil hebben,Der leken spieghel,Jans teesteye,None,None,4.709084e-03
11201,Daer hi elken loen sal gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,daar hij elk loen zullen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,1.320254e-02
14677,Ende elken loen zal gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,en elk loen zullen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,1.391839e-02
4707,Anders mans wijf onderwinden niet / Ende oec niet stelen wats ghesciet,Anderre wive onderwenden niet / Ende oec niet stelen wats ghesciet,ander man wijf onderwinden niet / en ook niet stelen wat geschieden,ander wijf onderwinden niet / en ook niet stelen wat geschieden,Der leken spieghel,Jans teesteye,None,None,2.554755e-02
9378,Maer anebeedden een calf / Datsi van ere ghieten daeden,Maer si aenbeedden een calf / Dat si goten van ere,maar aanbeden een kalf / dat zij van een gieten doen,maar zij aanbeden een kalf / dat zij gieten van een,Der leken spieghel,Jans teesteye,None,None,3.207626e-02
5333,Ende elken loen gheuen / Van dat hi hier heeft bedreuen,Hi elken loen sal gheven / Van dat hi hier heeft bedreven,en elk loen geven / van dat hij hier hebben bedrijven,hij elk loen zullen geven / van dat hij hier hebben bedrijven,Der leken spieghel,Jans teesteye,None,None,3.231414e-02


In [15]:
nearest_df = nearest_verses('Sidrac', 'Boec vander wraken',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/sidrac-wraken.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
221,Amen segt gemeenlike / Amen,Nu seght allegader Amen / Amen,amen zeggen gemeenlijk / amen,nu zeggen algader amen / amen,Sidrac,Boec vander wraken,None,None,0.153910
202,Ende ic bidde hem allen met trauwen / Die desen bouc selen scauwen,Daer om so biddic met trouwen / Hen die dit boec selen scouwen,en ik bidden hij al met trouw / die deze boek zullen schouwen,daar om zo bidden ik met trouw / zij die dit boek zullen schouwen,Sidrac,Boec vander wraken,None,None,0.157838
47,Soe quam my een boec ter hant / Daer ic in bescreven vant,MI quam een out boec in de hant / Daer ic in ghescreven vant,zo komen ik een boek ter hand / daar ik in beschrijven vinden,ik komen een oud boek in de hand / daar ik in schrijven vinden,Sidrac,Boec vander wraken,None,None,0.270186
203,Die desen bouc selen scauwen / Dat sij Gode bidden vor mi,Daer om so biddic met trouwen / Hen die dit boec selen scouwen,die deze boek zullen schouwen / dat zij NOU-P bidden voor ik,daar om zo bidden ik met trouw / zij die dit boek zullen schouwen,Sidrac,Boec vander wraken,None,None,0.297939
167,Dat God ende ghi allene / Enen sone hebt gemene,Dien ic hebbe allene / Met sinen Vader ghemene,dat NOU-P en gij alleen / een zoon hebben menen,dat ik hebben alleen / met zijn vader menen,Sidrac,Boec vander wraken,None,None,0.299863
107,Die beter es menichfout / Danne ghesteente zelver of gout,Ghesteinte silver ende gout / Ende diere juwele menichfout,die goed zijn menigvout / dan gesteente zelf of goud,gesteente zilver en goud / en die juweel menigvout,Sidrac,Boec vander wraken,None,None,0.302634
17,Ende troest van allen goeden rade / An leren mach vroech ende spade,Dese begheren te haren rade / Alle die liede vroech ende spade,en troosten van al goed raad / aan leren mogen vroeg en spade,deze begeren te haar raad / al die lieden vroeg en spade,Sidrac,Boec vander wraken,None,None,0.363732
13,Daer anders niet inne en leit / Dan der werelt ydelheit,Ende draghen in die stede / Deser werelt ydelhede,daar anders niet in ne liggen / dan de wereld ijdelheid,en dragen in die stede / deze wereld ijdelheid,Sidrac,Boec vander wraken,None,None,0.387950
37,Dat overlijt soe cortelike / Verdienen souden hemelrike,Die verdient hebben dat hemelrike / Oec salmen kinnen te waren,dat overlijd zo kortelijk / verdienen zullen hemelrijk,die verdienen hebben dat hemelrijk / ook zullen men kennen te zijn,Sidrac,Boec vander wraken,None,None,0.408332
179,Dus quamen in hare scone / Beide die heilege Geest ende die Sone,Cristus die heylighe Gods Sone / Spreect hier af harde scone,dus komen in haar scone / beide die heilig geest en die zoon,NOU-P die heilig NOU-P zoon / spreken hier af hard scone,Sidrac,Boec vander wraken,None,None,0.410291


In [16]:
nearest_df = nearest_verses('Sidrac', 'Jans teesteye',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/sidrac-teesteye.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
27,Aldus eest ende anders niet / Die de dinc te rechte besiet,Want die de dinc te rechte besiet / So en was Cristus leven anders niet,aldus zijn het en anders niet / die de ding te rechte bezien,want die de ding te rechte bezien / zo ne zijn NOU-P leven anders niet,Sidrac,Jans teesteye,None,None,0.190382
129,Doe hine metter gracie van Gode / Hadde bekeert te sinen ghebode,Die de gratie heeft te Gode / Men houde de X ghebode,toen hij hij met de gratie van NOU-P / hebben bekeren te zijn gebod,die de gratie hebben te NOU-P / men houden de tien gebod,Sidrac,Jans teesteye,None,None,0.365649
209,Al in die stat tAntwerpen / Wast dat ic dit translateerde,AL tote Andwerpen in die stat / Luste tween ghesellen dat,al in die stad te antwerpen / wassen dat ik dit translateren,al tot antwerpen in die stad / lust twee gezel dat,Sidrac,Jans teesteye,None,None,0.372528
167,Dat God ende ghi allene / Enen sone hebt gemene,Ende wanen dat ic allene / Die goede metten quaden mene,dat NOU-P en gij alleen / een zoon hebben menen,en wanen dat ik alleen / die goed met de kwaad menen,Sidrac,Jans teesteye,None,None,0.398759
221,Amen segt gemeenlike / Amen,Ewelijc in alder tijt Amen / DAvonture Ian wats dat,amen zeggen gemeenlijk / amen,eeuwelijk in al tijd amen / de avontuur NOU-P NOU-P dat,Sidrac,Jans teesteye,None,None,0.422503
17,Ende troest van allen goeden rade / An leren mach vroech ende spade,Ende pijnt daer om vroech ende spade / Ende alse hi dan te goede es comen,en troosten van al goed raad / aan leren mogen vroeg en spade,en pijnen daar om vroeg en spade / en als hij dan te goed zijn komen,Sidrac,Jans teesteye,None,None,0.448246
208,Verre van mi wille werpen / Al in die stat tAntwerpen,AL tote Andwerpen in die stat / Luste tween ghesellen dat,ver van ik willen werpen / al in die stad te antwerpen,al tot antwerpen in die stad / lust twee gezel dat,Sidrac,Jans teesteye,None,None,0.452776
188,Daer boven inden oversten trone / Daer die ingelen singen hoge,Alle die hemele ende die eerde / Ende daer boven die hoghe trone,daar boven in de overst troon / daar die engel zingen hoog,al die hemel en die aarde / en daar boven die hoog troon,Sidrac,Jans teesteye,None,None,0.455127
28,Die de dinc te rechte besiet / Die den tijt onledich maken,Want die de dinc te rechte besiet / So en was Cristus leven anders niet,die de ding te rechte bezien / die de tijd onledig maken,want die de ding te rechte bezien / zo ne zijn NOU-P leven anders niet,Sidrac,Jans teesteye,None,None,0.456363
145,Doen ic dit werc ierst began / Ende hebbe versleten nochtan,Doen kerstenheyt yrst began / Die keyser ende de heren nochtan,toen ik dit werk eerst beginnen / en hebben verslijten nochtan,toen kerstenheid eerst beginnen / die keizer en de heer nochtan,Sidrac,Jans teesteye,None,None,0.458656


In [17]:
nearest_df = nearest_verses('Melibeus', 'Dietsche doctrinale',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/melibeus-dietsche_doctrinale.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
3659,Seneca seghet noch dit woert / Dat nieman bat toe en hoert,Noch seit seneca dit woert / Dat niemanne bat toe en hoert,NOU-P zeggen nog dit woord / dat niemand bet toezenden ne hoeren,nog zeggen NOU-P dit woord / dat niemand bet toezenden ne hoeren,Melibeus,Dietsche doctrinale,None,None,0.000000
3245,Gheeft mi op die wrake / Ic sal lonen die sake,Hi sprect gheeft mi op die wrake / Ende ic sal lonen die sake,geven ik op die wrake / ik zullen lonen die sake,hij spreken geven ik op die wrake / en ik zullen lonen die sake,Melibeus,Dietsche doctrinale,None,None,0.040384
1884,Die van vele lieden ontsien es / Moet vele lieden ontsien weder,Vele lieden hi moet van dien / Vele lieden weder ontsien,die van veel lieden ontzien zijn / moeten veel lieden ontzien weder,veel lieden hij moeten van dat / veel lieden weder ontzien,Melibeus,Dietsche doctrinale,None,None,0.049200
2341,Ende so wien men ontsiet / Ende mach ghemint wesen niet,Soe wien datmen ontsiet / En mach ghemint wesen niet,en zo wie men ontzien / en mogen minnen wezen niet,zo wie dat men ontzien / ne mogen minnen wezen niet,Melibeus,Dietsche doctrinale,None,None,0.062377
1267,Haddic enen voete inden grave / Nochtan so woudic leren,Haddic enen voet int graf / Nochtan soudic willen leren,hebben ik een voet in de graf / nochtan zo willen ik leren,hebben ik een voet in het graf / nochtan zullen ik willen leren,Melibeus,Dietsche doctrinale,None,None,0.065264
1352,Die wise paeus Innocentius / Scrijft in sine boeke aldus,Die paus innocentius / Scrijft in sinen boeke aldus,die wijs paus NOU-P / schrijven in zijn beuk aldus,die paus NOU-P / schrijven in zijn beuk aldus,Melibeus,Dietsche doctrinale,None,None,0.076406
1026,Salomon seghet die hoedt sinen mont / Hoedt sijn ziele talre stont,Soe wie hoeden can sinen mont / Hoet sine ziele talre stont,NOU-P zeggen die hoeden zijn mond / hoeden zijn ziel te al stonde,zo wie hoeden kunnen zijn mond / hoeden zijn ziel te al stonde,Melibeus,Dietsche doctrinale,None,None,0.091893
2265,Wat si u segghen ende tonen / Hoedt u altoes jeghene honen,Watsi segghen ofte tonen / Hoedt u altoes jeghen honen,wat zij u zeggen en tonen / hoeden u altoos jegen hoon,watsie zeggen ofte tonen / hoeden u altoos jegen hoon,Melibeus,Dietsche doctrinale,None,None,0.107862
1466,Want stercmoedecheyt wet wale / Es een der doghet cardinale,Want hets ene doeght wet dat wale / Van iiij doeghden cardinale,want sterkmoedigheid wetten wel / zijn een de deugd kardinaal,want het zijn een deugd wetten dat wel / van vier deugd kardinaal,Melibeus,Dietsche doctrinale,None,None,0.110409
3215,Roepen ghemeynlijc over al / Datmen cracht met crachten weren sal,Roepen alle al ouer al / Datmen cracht met crachte weren sal,roepen gemeenlijk over al / dat men kracht met kracht weren zullen,roepen al al over al / dat men kracht met kracht weren zullen,Melibeus,Dietsche doctrinale,None,None,0.112590


In [18]:
nearest_df = nearest_verses('Brabantsche yeesten (B5)', 'Dietsche doctrinale',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/yeesten_5-dietsche_doctrinale.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
2334,Nochtan in allen manieren / Soe was hi hem goedertieren,Ontfermhertech ende goedertieren / Es hi in allen manieren,nochtan in al manier / zo zijn hij hij goedertieren,ontfermhartig en goedertieren / zijn hij in al manier,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.123100
844,Daer hi mochte vroech ende spade / Beide met rade ende met dade,Den bedroefden vroech ende spade / Troesten met rade ende met dade,daar hij mogen vroeg en spade / beide met raad en met daad,de bedroefde vroeg en spade / troosten met raad en met daad,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.150769
1050,Oec doetmen met wisen rade / Datmen met crachten niet en dade,Want men doet met wisen rade / Datmen met crachte niet en dade,ook doen men met wijs raad / dat men met kracht niet ne doen,want men doen met wijze raad / dat men met kracht niet ne doen,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.178547
3921,Souden al te nieute keren / Ende dat hi met Gode ende met eren,Ende hem te sulker neren keren / Die met gode si ende met eren,zullen al te niet keren / en dat hij met NOU-P en met eer,en hij te zulk neer keren / die met NOU-P zijn en met eer,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.191093
1144,Ende dat hi om sinen wille allene / Ende om ander dinc en ghene,Ende oec om ander dinc en ghene / Dan om dese viere allene,en dat hij om zijn wil alleen / en om ander ding ne gene,en ook om ander ding ne gene / dan om deze vuur alleen,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.240137
2983,Ende gherne sach si vrede / Eendrachticheit te waren,Van starcmoedecheiden / Van vreden ende eendrachtecheiden,en gaarne zien zij vrede / eendrachtigheid te zijn,van sterkmoedigheid / van vrede en eendrachtigheid,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.242410
1114,Een grooten pot van metale / Diene daer liet weet men wale,Nemt enen pot van motale / Ende enen anderen daer bi,een groot pot van metaal / die hij daar laten weten men wel,nemen een pot van metaal / en een ander daar bij,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.252541
3432,Ende oec swegher die ghelike / Edewaerts van Enghelant,En mach niet sijn haer ghelike / Ihesus sydrac seit des ghelike,en ook zweger die gelijke / edewaarts van NOU-P,ne mogen niet zijn haar gelijke / NOU-P NOU-P zeggen de gelijke,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.262960
1113,Wart daer vonden heb ic vernomen / Een grooten pot van metale,Nemt enen pot van motale / Ende enen anderen daer bi,worden daar vinden hebben ik vernemen / een groot pot van metaal,nemen een pot van metaal / en een ander daar bij,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.268511
3188,Van rechte den Almaenschen rike / Dat si daden groot onghelike,Metten hoeuerdeghen riken / Want si sere ongheliken,van rechte de Almaans rijk / dat zij doen groot ongelijk,met de hoevaardige rijk / want zij zeer ongelijk,Brabantsche yeesten (B5),Dietsche doctrinale,None,None,0.275700


In [19]:
nearest_df = nearest_verses('Brabantsche yeesten (B5)', 'Van den derden Eduwaert',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/yeesten_5-eduwaert.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
3297,Ende die grave van Hollant mede / Met consent van elker stede,Ende die grave van Hollant mede / Met consente van elker stede,en die graf van NOU-P mede / met consent van elk stede,en die graf van NOU-P mede / met consent van elk stede,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,0.000000e+00
3291,Daer al dlant an viel sere / Ieghen den grave sinen here,Daer al tlant an viel seere / Jegen den grave sinen heere,daar al dat land aan vallen zeer / jegen de graf zijn heer,daar al dat land aan vallen zeer / jegen de graf zijn heer,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,0.000000e+00
3046,Ende die grave van Gulke met / Dies beide hadden te bet,Ende die grave van Gulken met / Dies beide hadden te bet,en die graf van NOU-P met / dies beide hebben te bet,en die graf van NOU-P met / dies beide hebben te bet,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,0.000000e+00
3303,Ieghen elc lant ende elken here / Ende dit soude dueren emmermere,Jegen elc lant ende elken heere / Ende dit soude duren emmermeere,jegen elk land en elk heer / en dit zullen duren immermeer,jegen elk land en elk heer / en dit zullen duren immermeer,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.110223e-16
3260,Doen die heren hoerden clare / Dat die hertoghe noch was dare,Doe die heeren hoerden claer / Dat die hertoge noch waere daer,toen die heer hoeren klaar / dat die hertog nog zijn daar,toen die heer hoeren klaar / dat die hertog nog zijn daar,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.110223e-16
3047,Dies beide hadden te bet / Want si in dit bedrijf tsamen,Dies beide hadden te bet / Want sij in dit bedrijf te samen,dies beide hebben te bet / want zij in dit bedrijf samen,dies beide hebben te bet / want zij in dit bedrijf te samen,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.129563e-02
3259,Oft opt velt te samen sterven / Doen die heren hoerden clare,Oft opt velt tesamen sterven / Doe die heeren hoerden claer,of op het veld te samen sterven / toen die heer hoeren klaar,of op het veld samen sterven / toen die heer hoeren klaar,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.325845e-02
3050,Want deen wil heten hertoghe / Ende dander marcgrave hoghe,Want deene wilt heeten hertoge / Ende dandre een marcgrave hoge,want de een willen heten hertog / en de ander markgraaf hoog,want de een willen heten hertog / en de ander een markgraaf hoog,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.414181e-02
3301,Dat elc soude anderen wale / Helpen houden sine pale,Ende elc soude andren wale / Helpen houden sine pale,dat elk zullen ander wel / helpen houden zijn paal,en elk zullen ander wel / helpen houden zijn paal,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,1.473947e-02
3450,Soe en soude kerstenhede / Die scade nemmermeer verwinnen,Sone soude dat kerstenhede / De scade nemmermeer verwinnen,zo ne zullen kerstenheid / die schade nimmermeer verwinnen,zo ne zullen dat kerstenheid / de schade nimmermeer verwinnen,Brabantsche yeesten (B5),Van den derden Eduwaert,None,None,2.631093e-02


In [20]:
nearest_df = nearest_verses('Brabantsche yeesten (B5)', 'Korte kroniek van Brabant (lange versie)',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/yeesten_5-korte_kroniek.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
3008,Ten Minderbroedren leit si begraven / Te Brusele met groter haven,Keiser Kaerle was begraven / Tot Aken met groter haven,ten minderbroederen liggen zij begraven / te NOU-P met groot haven,keizer NOU-P zijn begraven / tot NOU-P met groot haven,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.242649
560,Hertoghe Jan sijn ouder vader / Was doot eer sconincs vader,Die hi ende siin vader Puppiin / Ende Kaerle Marteel die oude vader siin,hertog NOU-P zijn oud vader / zijn dood eer de koning vader,die hij en zijn vader pupiin / en NOU-P NOU-P die oud vader zijn,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.254300
2290,Alsoe hart in desen doene / Ende met hem die Brabantsoene,Ende die Oosterse Brabansoene / Quamen tot desen hoghen doene,alzo hard in deze doen / en met hij die Brabantsoen,en die oosters Brabantsoen / komen tot deze hoog doen,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.265807
677,Ontghelden dat sijn ouder vader / Ende sijn vader hadde ghedaen,Die hi ende siin vader Puppiin / Ende Kaerle Marteel die oude vader siin,ontgelden dat zijn oud vader / en zijn vader hebben doen,die hij en zijn vader pupiin / en NOU-P NOU-P die oud vader zijn,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.280535
3996,Doemen screef Ons Heren jaer / Dertien hondert ende veertach,Doemen screef die iare ons Heren / Dertien hondert ende xij mede,toen men schrijven ons heer jaar / dertien honderd een veertig,toen men schrijven die jaar ons heer / dertien honderd een tweeëntwintigste mede,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.282003
622,Dede hien nemen metter vaert / Ende voeren te Ghenapie waert,Ende Montaguut an siinre vaert / Daer na voer hi tot Riemen waert,doen hij hij nemen met de vaart / en voeren te NOU-P waart,en NOU-P aan zijn vaart / daar na voeren hij tot NOU-P waart,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.289993
1662,Ieghen den hertoghe van Brabant / Trocken si oec al te hant,Jegen die hertogen van Brabant / Die Oosteriken hadden in hant,jegen de hertog van NOU-P / trekken zij ook al te hand,jegen die hertog van NOU-P / die oosterrijk hebben in hand,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.319653
2494,Ende lieten riden op die waerde / Ende dat hi keerde metter vaerde,Met eren ende met groter waerde / Ende keerde weder siinre vaerde,en laten rijden op die waarde / en dat hij keren met de vaart,met eer en met groot waarde / en keren woeden zijn vaart,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.323736
466,Maer den grave daden si scade groot / Want sijn liede bleven daer doot,Ende dede Hughen scade groot / Dat veel der siinre bleven doot,maar de graf doen zij schade groot / want zijn lieden blijven daar dood,en doen NOU-P schade groot / dat veel de zinder blijven dood,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.327178
165,Dus soe wies daer een ghevecht / Tusschen Tricht ende dien here,Tusschen Ludike ende Maestrecht / Hi verwan in een ghevecht,dus zo wassen daar een gevecht / tussen NOU-P en dat heer,tussen NOU-P en NOU-P / hij verwinnen in een gevecht,Brabantsche yeesten (B5),Korte kroniek van Brabant (lange versie),None,None,0.327280


In [21]:
nearest_df = nearest_verses('Sidrac', 'Jans teesteye',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/sidrac-teesteye.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
27,Aldus eest ende anders niet / Die de dinc te rechte besiet,Want die de dinc te rechte besiet / So en was Cristus leven anders niet,aldus zijn het en anders niet / die de ding te rechte bezien,want die de ding te rechte bezien / zo ne zijn NOU-P leven anders niet,Sidrac,Jans teesteye,None,None,0.190382
129,Doe hine metter gracie van Gode / Hadde bekeert te sinen ghebode,Die de gratie heeft te Gode / Men houde de X ghebode,toen hij hij met de gratie van NOU-P / hebben bekeren te zijn gebod,die de gratie hebben te NOU-P / men houden de tien gebod,Sidrac,Jans teesteye,None,None,0.365649
209,Al in die stat tAntwerpen / Wast dat ic dit translateerde,AL tote Andwerpen in die stat / Luste tween ghesellen dat,al in die stad te antwerpen / wassen dat ik dit translateren,al tot antwerpen in die stad / lust twee gezel dat,Sidrac,Jans teesteye,None,None,0.372528
167,Dat God ende ghi allene / Enen sone hebt gemene,Ende wanen dat ic allene / Die goede metten quaden mene,dat NOU-P en gij alleen / een zoon hebben menen,en wanen dat ik alleen / die goed met de kwaad menen,Sidrac,Jans teesteye,None,None,0.398759
221,Amen segt gemeenlike / Amen,Ewelijc in alder tijt Amen / DAvonture Ian wats dat,amen zeggen gemeenlijk / amen,eeuwelijk in al tijd amen / de avontuur NOU-P NOU-P dat,Sidrac,Jans teesteye,None,None,0.422503
17,Ende troest van allen goeden rade / An leren mach vroech ende spade,Ende pijnt daer om vroech ende spade / Ende alse hi dan te goede es comen,en troosten van al goed raad / aan leren mogen vroeg en spade,en pijnen daar om vroeg en spade / en als hij dan te goed zijn komen,Sidrac,Jans teesteye,None,None,0.448246
208,Verre van mi wille werpen / Al in die stat tAntwerpen,AL tote Andwerpen in die stat / Luste tween ghesellen dat,ver van ik willen werpen / al in die stad te antwerpen,al tot antwerpen in die stad / lust twee gezel dat,Sidrac,Jans teesteye,None,None,0.452776
188,Daer boven inden oversten trone / Daer die ingelen singen hoge,Alle die hemele ende die eerde / Ende daer boven die hoghe trone,daar boven in de overst troon / daar die engel zingen hoog,al die hemel en die aarde / en daar boven die hoog troon,Sidrac,Jans teesteye,None,None,0.455127
28,Die de dinc te rechte besiet / Die den tijt onledich maken,Want die de dinc te rechte besiet / So en was Cristus leven anders niet,die de ding te rechte bezien / die de tijd onledig maken,want die de ding te rechte bezien / zo ne zijn NOU-P leven anders niet,Sidrac,Jans teesteye,None,None,0.456363
145,Doen ic dit werc ierst began / Ende hebbe versleten nochtan,Doen kerstenheyt yrst began / Die keyser ende de heren nochtan,toen ik dit werk eerst beginnen / en hebben verslijten nochtan,toen kerstenheid eerst beginnen / die keizer en de heer nochtan,Sidrac,Jans teesteye,None,None,0.458656


In [22]:
nearest_df = nearest_verses('Sidrac', 'Melibeus',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.to_excel('../figures/sidrac-melibeus.xlsx')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,intertext1,intertext2,distance
221,Amen segt gemeenlike / Amen,Nu segt Amen ghemeynlike / AMEN,amen zeggen gemeenlijk / amen,nu zeggen amen gemeenlijk / amen,Sidrac,Melibeus,None,None,0.017344
107,Die beter es menichfout / Danne ghesteente zelver of gout,Dat wijf es beter menechfout / Dan ghesteynte silver of gout,die goed zijn menigvout / dan gesteente zelf of goud,dat wijf zijn goed menigvout / dan gesteente zilver of goud,Sidrac,Melibeus,None,None,0.157572
220,Ende na dit leven hemelrike / Amen segt gemeenlike,Gheloeft si God van hemelrike / Nu segt Amen ghemeynlike,en na dit leven hemelrijk / amen zeggen gemeenlijk,geloven zij NOU-P van hemelrijk / nu zeggen amen gemeenlijk,Sidrac,Melibeus,None,None,0.224553
17,Ende troest van allen goeden rade / An leren mach vroech ende spade,Daer om in ghedeelden rade / Salmen volghen vroech ende spade,en troosten van al goed raad / aan leren mogen vroeg en spade,daar om in gedeeld raad / zullen men volgen vroeg en spade,Sidrac,Melibeus,None,None,0.366495
35,Want die tijt es ons ghegeuen / Omme dat wy in dit leven,Al dat ons in dit leven / Van boven wert ghegheven,want die tijd zijn ons geven / om dat wij in dit leven,al dat ons in dit leven / van boven worden geven,Sidrac,Melibeus,None,None,0.385236
171,Ende die selve Sone es God / Seker hy es herde sot,Nochtan en sidi sot niet / Want die hem selven sot siet,en die zelf zoon zijn NOU-P / zeker hij zijn hard zot,nochtan ne zijn gij zot niet / want die hij zelf zot zien,Sidrac,Melibeus,None,None,0.419579
108,Danne ghesteente zelver of gout / Want vele beter es wijshede,Dat wijf es beter menechfout / Dan ghesteynte silver of gout,dan gesteente zelf of goud / want veel goed zijn wijsheid,dat wijf zijn goed menigvout / dan gesteente zilver of goud,Sidrac,Melibeus,None,None,0.434223
219,Ende sine helige minne mede / Ende na dit leven hemelrike,Ende God na dit corte leven / Sal hem hemelrike gheven,en zijn heilig minne mede / en na dit leven hemelrijk,en NOU-P na dit kort leven / zullen hij hemelrijk geven,Sidrac,Melibeus,None,None,0.436349
158,Die my int herte niet en ginc / Hebbic al gelaten uut,En hebbic niet in mi / Al woudic al laten varen,die ik in het hart niet ne gaan / hebben ik al laten uiten,ne hebben ik niet in ik / al willen ik al laten varen,Sidrac,Melibeus,None,None,0.467414
209,Al in die stat tAntwerpen / Wast dat ic dit translateerde,Ghetracteert uten latine / Al tAntwerpen in die poert,al in die stad te antwerpen / wassen dat ik dit translateren,trakteren uit dat NOU-P / al te antwerpen in die port,Sidrac,Melibeus,None,None,0.476280
